In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import plotly.express as px
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Lendo Base de dados
Base_dados = pd.read_csv("https://raw.githubusercontent.com/rafaelduria/Clustering/main/novo.csv", sep=';')
#Mostrar todas colunas
pd.set_option('display.max_columns', None)
#Mostrar todas linhas
pd.set_option('display.max_rows', None)

In [ ]:
#Visualizando DataFrame
Base_dados.head()

,Material,Venda_pallet,Embalagem,Quantidade_de_cxs,Data,Quantidade,Peso,Frequencia
0,3985,"0,030381944",N1,35,44713,350,7,7
1,3985,"0,034722222",N1,40,44562,400,8,5
2,3985,"0,037326389",N1,43,44866,430,"8,6",4
3,3985,"0,046006944",N1,53,44774,530,"10,6",5
4,3985,"0,061631944",N1,71,44743,710,"14,2",5


In [ ]:
#Convertendo string para float
Base_dados['Venda_pallet'] = Base_dados['Venda_pallet'].str.replace(',','.').astype(float)

In [ ]:
#Preenchendo valores nulos com zero
Base_dados = Base_dados.fillna({'Venda_pallet':0,'Quantidade_de_cxs':0})

In [ ]:
#Desconsiderando valores igual zero ou menor do que 0
Base_dados = Base_dados.loc[(Base_dados['Venda_pallet'] > 0) & (Base_dados['Frequencia'] > 0) & (Base_dados['Quantidade_de_cxs'] > 0)]

In [ ]:
#Selecionando apenas colunas " Material, Venda_pallet,Frenquencia"
Base_dados = Base_dados[['Material','Venda_pallet','Frequencia']]
Base_dados.head()

,Material,Venda_pallet,Frequencia
0,3985,0.030382,7
1,3985,0.034722,5
2,3985,0.037326,4
3,3985,0.046007,5
4,3985,0.061632,5


In [ ]:
#Agrupando dados e removendo duplicatas
Base_dados['Venda_pallet'] = Base_dados.groupby('Material')["Venda_pallet"].transform(np.mean).round(2)
Base_dados['Frequencia'] = Base_dados.groupby('Material')["Frequencia"].transform(np.mean).round(2)
Base_dados = Base_dados.drop_duplicates()
Base_dados.head()

,Material,Venda_pallet,Frequencia
0,3985,0.14,7.45
11,3986,0.20,2.27
22,3987,0.01,1.00
33,3988,0.12,5.18
44,3989,0.01,1.00


In [ ]:
#Criando uma base uma copia da Base_dados
Base_dados_para_clusterizacao = Base_dados.copy()

In [ ]:
#removendo Coluna Material
Base_dados_para_clusterizacao.drop(['Material'], axis=1, inplace=True)

In [ ]:
#Criando outro base de dados para novos algoritmos
df = Base_dados_para_clusterizacao.copy()

##Normalizando e Rodando Kmeans

In [ ]:
#Normalizando dados
values = Normalizer().fit_transform(Base_dados_para_clusterizacao.values)

In [ ]:
#escolhendo parâmetros para Kmeans

""" n_clusters =  número de cluster """
""" max_iter = número maximo de interações"""

"""
algorithm =  seleciona entre três formatos de cálculo das distâncias:

   full: padrão dos algoritmos de k-means que calcula a distância de todos os  pontos com relação ao centro;

   elkan: converge para os mesmos resultados, mas reduz, a partir de  triangulações,a quantidade de distâncias calculadas, eliminando muitas etapas desnecessárias/redundantes;

   auto: recorre ao método convencional para conjuntos de dados esparsos, em que o algoritmo elkan implementado é incapaz de convergir, e ao método elkan para conjuntos de dados densos.
"""

kmeans = KMeans(
                n_clusters = 10,
                max_iter = 300,
                n_init = 'auto',
                algorithm = 'elkan'
                )


y_pred = kmeans.fit_predict(Base_dados_para_clusterizacao.values)
labels = kmeans.labels_

##Metricas

In [ ]:
"""Silhouette
MAIOR QUE 0, coeficiente vai de -1 até 1.
Então, a partir do momento que temos valor positivo,conseguimos ter uma ideia se o cluster está bom de acordo com esta métrica ou não.
"""

silhouette = metrics.silhouette_score(values, labels, metric='euclidean')
print('Silhouette_score {:.3f}'.format(silhouette))


#Davies Bouldin, MAIS PROXIMO DE 0, melhor.
dbs = metrics.davies_bouldin_score(values, labels)
print('Davies_bouldin_score {:.3f}'.format(dbs))

#Calinski,MAIS ALTO melhor.
calinski = metrics.calinski_harabasz_score(values, labels)
print('Calinski_harabasz_score {:.3f}'.format(calinski))


Silhouette_score -0.465
Davies_bouldin_score 19.243
Calinski_harabasz_score 63.929


In [ ]:
#Incluindo coluna cluster no dataframe Base_dados_para_clusterizacao
Base_dados_para_clusterizacao['cluster'] = labels

In [ ]:
#Criando coluna cluster no dataframe material_base

Base_dados['cluster'] = Base_dados_para_clusterizacao['cluster']

In [ ]:
Base_dados.to_excel('base_dados.xlsx')

In [ ]:
#Criando agrupamento de frequência por clusters
Frequencia_groupby = Base_dados_para_clusterizacao[['cluster','Frequencia']].groupby(by='cluster')
Frequencia_groupby = Frequencia_groupby.agg(func=['count','min','mean','max']).round(2)
Frequencia_groupby = Frequencia_groupby.stack(0)
Frequencia_groupby.sort_values(['mean'], ascending=False, inplace=True)
Frequencia_groupby.reset_index(inplace=True)
Frequencia_groupby.rename(columns={'level_1': 'Freq_Col'}, inplace = True)
Frequencia_groupby.drop(['count'], axis=1, inplace=True)
Frequencia_groupby.head(2)

,cluster,Freq_Col,min,mean,max
0,2,Frequencia,445.00,525.69,618.09
1,8,Frequencia,316.82,362.70,414.45


In [ ]:
#Criando agrupamento de Pallets por clusters
Venda_pallet_groupby = Base_dados_para_clusterizacao[['cluster','Venda_pallet']].groupby(by='cluster')
Venda_pallet_groupby = Venda_pallet_groupby.agg(func=['count','min','mean','max']).round(2)
Venda_pallet_groupby = Venda_pallet_groupby.stack(0)
Venda_pallet_groupby.sort_values(['mean'], ascending=False, inplace=True)
Venda_pallet_groupby.reset_index(inplace=True)
Venda_pallet_groupby.rename(columns={'level_1': 'Pallet_Col'}, inplace = True)
Venda_pallet_groupby.head(2)

,cluster,Pallet_Col,count,min,mean,max
0,2,Venda_pallet,11,13.36,26.39,56.30
1,8,Venda_pallet,18,6.05,15.36,30.22


In [ ]:
#Juntando/Merge Frquencia com venda_pallet
Junção_Frequencia_groupby_Venda_pallet_groupby = Frequencia_groupby.merge(Venda_pallet_groupby, on='cluster', how='left')
Junção_Frequencia_groupby_Venda_pallet_groupby.head(2)

,cluster,Freq_Col,min_x,mean_x,max_x,Pallet_Col,count,min_y,mean_y,max_y
0,2,Frequencia,445.00,525.69,618.09,Venda_pallet,11,13.36,26.39,56.30
1,8,Frequencia,316.82,362.70,414.45,Venda_pallet,18,6.05,15.36,30.22


In [ ]:
#Ordenando selecionando e ordenando colunas
#Tabela final para análise de agrupamentos.
Junção_Frequencia_groupby_Venda_pallet_groupby = Junção_Frequencia_groupby_Venda_pallet_groupby.iloc[:,[0,6,1,2,4,3,5,7,9,8]]
Junção_Frequencia_groupby_Venda_pallet_groupby

,cluster,count,Freq_Col,min_x,max_x,mean_x,Pallet_Col,min_y,max_y,mean_y
0,2,11,Frequencia,445.00,618.09,525.69,Venda_pallet,13.36,56.30,26.39
1,8,18,Frequencia,316.82,414.45,362.70,Venda_pallet,6.05,30.22,15.36
2,5,44,Frequencia,238.00,302.55,267.27,Venda_pallet,3.23,50.12,9.69
3,3,58,Frequencia,176.45,234.64,204.10,Venda_pallet,1.31,43.81,6.98
4,9,122,Frequencia,122.25,175.55,147.50,Venda_pallet,0.65,10.77,3.69
5,1,304,Frequencia,78.45,121.45,95.79,Venda_pallet,0.26,40.21,2.49
6,7,508,Frequencia,48.43,78.20,60.93,Venda_pallet,0.13,12.24,1.22
7,4,899,Frequencia,26.27,48.36,35.65,Venda_pallet,0.06,17.49,0.68
8,6,1829,Frequencia,10.27,26.20,16.62,Venda_pallet,0.03,6.77,0.37
9,0,4210,Frequencia,1.00,10.27,3.90,Venda_pallet,0.01,13.40,0.24


In [ ]:
# posições estoque  em função da Frequência
#     count
#803 = 80                        (Melhor depósito - mais perto da expedição)
#801 e 802 =  637                             |
#806 = 467                                    |
#810 = 2666                                   |
#804  = 2485                                  |
#820 = 1945                                   V
#807 = 2595                      (Pior depósito em termos de distância da expedição)
# soma = 10875
# count = É a quantidade de itens dentro do cluster
# Um caminhão Bau = 22 pallets

In [ ]:
#Grafico Plotly de pontinhos
KMeans = px.scatter(Base_dados, x=Base_dados.Venda_pallet, y=Base_dados.Frequencia, color='cluster',title='KMeans')
KMeans.update_layout(width=1400, height=900,title_font_color='black',title_font_size=24)
KMeans.show()
KMeans.write_html('KMeans.html')

###Testando outro algoritmo
#####Mini Batch K-Means

In [ ]:
#importando bibliotecas
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler

ssc = StandardScaler()

#definindo e aplicando nos dados a normalização, de modo a evitar problemas por discrepancia dimensional

values = ssc.fit_transform(df.values)

#definindo o modelo de clustering que iremos utilizar
mb = MiniBatchKMeans(n_clusters=10,init='k-means++', random_state=0, batch_size=10000)

#aplicando o algoritmo nos dados normalizados
labels = mb.fit_predict(df.values)
df['labels'] = labels.astype(str)


silhouette = metrics.silhouette_score(values, labels, metric='euclidean')
print('Silhouette_score {:.3f}'.format(silhouette))


#Davies Bouldin, MAIS PROXIMO DE 0, melhor.
dbs = metrics.davies_bouldin_score(values, labels)
print('Davies_bouldin_score {:.3f}'.format(dbs))

#Calinski,MAIS ALTO melhor.
calinski = metrics.calinski_harabasz_score(values, labels)
print('Calinski_harabasz_score {:.3f}'.format(calinski))


#Criando agrupamento de frequência por labelss
Frequencia_groupby = df[['labels','Frequencia']].groupby(by='labels')
Frequencia_groupby = Frequencia_groupby.agg(func=['count','min','mean','max']).round(2)
Frequencia_groupby = Frequencia_groupby.stack(0)
Frequencia_groupby.sort_values(['mean'], ascending=False, inplace=True)
Frequencia_groupby.reset_index(inplace=True)
Frequencia_groupby.rename(columns={'level_1': 'Freq_Col'}, inplace = True)
Frequencia_groupby.drop(['count'], axis=1, inplace=True)

#Criando agrupamento de Pallets por labelss
Venda_pallet_groupby = df[['labels','Venda_pallet']].groupby(by='labels')
Venda_pallet_groupby = Venda_pallet_groupby.agg(func=['count','min','mean','max']).round(2)
Venda_pallet_groupby = Venda_pallet_groupby.stack(0)
Venda_pallet_groupby.sort_values(['mean'], ascending=False, inplace=True)
Venda_pallet_groupby.reset_index(inplace=True)
Venda_pallet_groupby.rename(columns={'level_1': 'Pallet_Col'}, inplace = True)

#Juntando/Merge Frquencia com venda_pallet
Junção_Frequencia_groupby_Venda_pallet_groupby = Frequencia_groupby.merge(Venda_pallet_groupby, on='labels', how='left')

#Tabela final para análise de agrupamentos.
Junção_Frequencia_groupby_Venda_pallet_groupby = Junção_Frequencia_groupby_Venda_pallet_groupby.iloc[:,[0,6,1,2,4,3,5,7,9,8]]


#Grafico Plotly de pontinhos
Mini_Batch_K_Means = px.scatter(df, x=df.Venda_pallet, y=df.Frequencia, color='labels',title='Mini Batch K-Means')
Mini_Batch_K_Means.update_layout(width=1400, height=900,title_font_color='black',title_font_size=24)
Mini_Batch_K_Means.show()
Mini_Batch_K_Means.write_html('Mini Batch K-Means.html')


Silhouette_score 0.494
Davies_bouldin_score 1.627
Calinski_harabasz_score 3225.182


In [ ]:
#Grafico Plotly de pontinhos
Mini_Batch_K_Means = px.scatter(df, x=df.Venda_pallet, y=df.Frequencia, color='labels',title='Mini Batch K-Means', template='plotly_dark')
Mini_Batch_K_Means.update_layout(width=1400, height=700,title_font_color='white',title_font_size=24)
Mini_Batch_K_Means.show()
Mini_Batch_K_Means.write_html('Mini Batch K-Means.html')

In [ ]:
#Mini Batch K-Means
Junção_Frequencia_groupby_Venda_pallet_groupby

,labels,count,Freq_Col,min_x,max_x,mean_x,Pallet_Col,min_y,max_y,mean_y
0,2,27,Frequencia,327.36,618.09,432.38,Venda_pallet,9.55,56.30,20.14
1,4,82,Frequencia,198.27,320.18,245.16,Venda_pallet,1.31,50.12,9.13
2,9,113,Frequencia,134.18,198.91,160.73,Venda_pallet,0.65,13.54,3.98
3,1,201,Frequencia,91.27,133.91,108.12,Venda_pallet,0.26,40.21,3.10
4,7,394,Frequencia,59.64,91.00,73.64,Venda_pallet,0.19,6.95,1.48
5,3,662,Frequencia,35.55,59.55,46.05,Venda_pallet,0.11,12.24,0.92
6,6,1058,Frequencia,18.90,35.57,26.06,Venda_pallet,0.04,17.49,0.52
7,8,1664,Frequencia,8.45,18.89,12.92,Venda_pallet,0.01,9.01,0.31
8,5,1352,Frequencia,3.89,8.45,5.99,Venda_pallet,0.01,6.73,0.19
9,0,2450,Frequencia,1.00,3.86,1.84,Venda_pallet,0.01,13.40,0.26


###Testando outro algoritmo
####DBScan

In [ ]:
#importando bibliotecas
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

ssc = StandardScaler()

#definindo e aplicando nos dados a normalização, de modo a evitar problemas por discrepancia dimensional
values = ssc.fit_transform(df.values)

#definindo o modelo de clustering que iremos utilizar
db = DBSCAN(eps=1,min_samples=5)

#aplicando o algoritmo nos dados normalizados
labels = db.fit_predict(df.values)
df['labels'] = labels.astype(str)

silhouette = metrics.silhouette_score(values, labels, metric='euclidean')
print('Silhouette_score {:.3f}'.format(silhouette))

#Davies Bouldin, MAIS PROXIMO DE 0, melhor.
dbs = metrics.davies_bouldin_score(values, labels)
print('Davies_bouldin_score {:.3f}'.format(dbs))

#Calinski,MAIS ALTO melhor.
calinski = metrics.calinski_harabasz_score(values, labels)
print('Calinski_harabasz_score {:.3f}'.format(calinski))

#Criando agrupamento de frequência por labelss
Frequencia_groupby = df[['labels','Frequencia']].groupby(by='labels')
Frequencia_groupby = Frequencia_groupby.agg(func=['count','min','mean','max']).round(2)
Frequencia_groupby = Frequencia_groupby.stack(0)
Frequencia_groupby.sort_values(['mean'], ascending=False, inplace=True)
Frequencia_groupby.reset_index(inplace=True)
Frequencia_groupby.rename(columns={'level_1': 'Freq_Col'}, inplace = True)
Frequencia_groupby.drop(['count'], axis=1, inplace=True)

#Criando agrupamento de Pallets por labelss
Venda_pallet_groupby = df[['labels','Venda_pallet']].groupby(by='labels')
Venda_pallet_groupby = Venda_pallet_groupby.agg(func=['count','min','mean','max']).round(2)
Venda_pallet_groupby = Venda_pallet_groupby.stack(0)
Venda_pallet_groupby.sort_values(['mean'], ascending=False, inplace=True)
Venda_pallet_groupby.reset_index(inplace=True)
Venda_pallet_groupby.rename(columns={'level_1': 'Pallet_Col'}, inplace = True)

#Juntando/Merge Frquencia com venda_pallet
Junção_Frequencia_groupby_Venda_pallet_groupby = Frequencia_groupby.merge(Venda_pallet_groupby, on='labels', how='left')

#Tabela final para análise de agrupamentos.
Junção_Frequencia_groupby_Venda_pallet_groupby = Junção_Frequencia_groupby_Venda_pallet_groupby.iloc[:,[0,6,1,2,4,3,5,7,9,8]]


#Grafico Plotly de pontinhos
fig = px.scatter(df, x=df.Venda_pallet, y=df.Frequencia, color='labels')
fig.show()

Silhouette_score 0.712
Davies_bouldin_score 1.603
Calinski_harabasz_score 1467.884


In [ ]:
Junção_Frequencia_groupby_Venda_pallet_groupby

,labels,count,Freq_Col,min_x,max_x,mean_x,Pallet_Col,min_y,max_y,mean_y
0,-1,379,Frequencia,1.00,618.09,164.41,Venda_pallet,0.26,56.30,6.62
1,6,9,Frequencia,115.27,117.09,116.42,Venda_pallet,0.60,1.91,1.35
2,12,5,Frequencia,113.27,114.73,113.97,Venda_pallet,2.03,2.63,2.29
3,10,5,Frequencia,112.00,113.00,112.55,Venda_pallet,3.34,4.84,3.90
4,8,16,Frequencia,109.45,113.27,110.94,Venda_pallet,0.51,2.45,1.50
5,9,14,Frequencia,105.45,107.45,106.33,Venda_pallet,0.67,3.29,2.13
6,11,12,Frequencia,102.91,104.50,103.81,Venda_pallet,1.11,3.46,2.61
7,7,60,Frequencia,91.45,101.18,95.66,Venda_pallet,0.33,3.50,1.65
8,5,371,Frequencia,59.64,91.00,73.39,Venda_pallet,0.19,3.48,1.29
9,4,646,Frequencia,35.64,59.55,46.04,Venda_pallet,0.11,3.44,0.83


###Testando outro algoritimo
####BIRCH

In [ ]:
#importando bibliotecas
from sklearn.cluster import Birch
from sklearn.preprocessing import StandardScaler

ssc = StandardScaler()

#definindo e aplicando nos dados a normalização, de modo a evitar problemas por discrepancia dimensional
values = ssc.fit_transform(df.values)


#definindo o modelo de clustering que iremos utilizar
model = Birch(n_clusters=10)

#aplicando o algoritmo nos dados normalizados
labels = model.fit_predict(df.values)
df['labels'] = labels.astype(str)

silhouette = metrics.silhouette_score(values, labels, metric='euclidean')
print('Silhouette_score {:.3f}'.format(silhouette))


#Davies Bouldin, MAIS PROXIMO DE 0, melhor.
dbs = metrics.davies_bouldin_score(values, labels)
print('Davies_bouldin_score {:.3f}'.format(dbs))

#Calinski,MAIS ALTO melhor.
calinski = metrics.calinski_harabasz_score(values, labels)
print('Calinski_harabasz_score {:.3f}'.format(calinski))



#Criando agrupamento de frequência por labelss
Frequencia_groupby = df[['labels','Frequencia']].groupby(by='labels')
Frequencia_groupby = Frequencia_groupby.agg(func=['count','min','mean','max']).round(2)
Frequencia_groupby = Frequencia_groupby.stack(0)
Frequencia_groupby.sort_values(['mean'], ascending=False, inplace=True)
Frequencia_groupby.reset_index(inplace=True)
Frequencia_groupby.rename(columns={'level_1': 'Freq_Col'}, inplace = True)
Frequencia_groupby.drop(['count'], axis=1, inplace=True)

#Criando agrupamento de Pallets por labelss
Venda_pallet_groupby = df[['labels','Venda_pallet']].groupby(by='labels')
Venda_pallet_groupby = Venda_pallet_groupby.agg(func=['count','min','mean','max']).round(2)
Venda_pallet_groupby = Venda_pallet_groupby.stack(0)
Venda_pallet_groupby.sort_values(['mean'], ascending=False, inplace=True)
Venda_pallet_groupby.reset_index(inplace=True)
Venda_pallet_groupby.rename(columns={'level_1': 'Pallet_Col'}, inplace = True)

#Juntando/Merge Frquencia com venda_pallet
Junção_Frequencia_groupby_Venda_pallet_groupby = Frequencia_groupby.merge(Venda_pallet_groupby, on='labels', how='left')

#Tabela final para análise de agrupamentos.
Junção_Frequencia_groupby_Venda_pallet_groupby = Junção_Frequencia_groupby_Venda_pallet_groupby.iloc[:,[0,6,1,2,4,3,5,7,9,8]]


#Grafico Plotly de pontinhos
Birch = px.scatter(df, x=df.Venda_pallet, y=df.Frequencia, color='labels',title='Birch')
Birch.update_layout(width=1400, height=900,title_font_color='black',title_font_size=24)
Birch.show()
Birch.write_html('Birch.html')

Silhouette_score 0.482
Davies_bouldin_score 2.281
Calinski_harabasz_score 1803.443


In [ ]:
Junção_Frequencia_groupby_Venda_pallet_groupby

,labels,count,Freq_Col,min_x,max_x,mean_x,Pallet_Col,min_y,max_y,mean_y
0,5,6,Frequencia,534.64,618.09,571.92,Venda_pallet,13.36,56.30,28.93
1,1,12,Frequencia,379.27,496.45,428.79,Venda_pallet,9.55,33.92,19.56
2,8,11,Frequencia,316.82,365.55,339.47,Venda_pallet,6.05,30.22,14.39
3,0,47,Frequencia,233.91,302.55,265.16,Venda_pallet,3.23,50.12,9.68
4,6,52,Frequencia,176.45,231.73,203.84,Venda_pallet,1.31,43.81,7.02
5,3,106,Frequencia,130.82,179.36,152.41,Venda_pallet,0.65,10.77,3.80
6,9,107,Frequencia,102.91,127.86,114.01,Venda_pallet,0.51,11.92,2.94
7,4,372,Frequencia,64.36,107.55,81.79,Venda_pallet,0.22,40.21,1.97
8,7,693,Frequencia,37.55,65.73,49.57,Venda_pallet,0.11,12.24,0.95
9,2,6597,Frequencia,1.00,38.36,9.75,Venda_pallet,0.01,17.49,0.31


###Testando outro algoritimo
####Agglomerative Clustering

In [ ]:
#importando bibliotecas
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler

ssc = StandardScaler()

#definindo e aplicando nos dados a normalização, de modo a evitar problemas por discrepancia dimensional
values = ssc.fit_transform(df.values)

#definindo o modelo de clustering que iremos utilizar
agg = AgglomerativeClustering(n_clusters=10)

#aplicando o algoritmo nos dados normalizados
labels = agg.fit_predict(df.values)
df['labels'] = labels.astype(str)


silhouette = metrics.silhouette_score(values, labels, metric='euclidean')
print('Silhouette_score {:.3f}'.format(silhouette))


#Davies Bouldin, MAIS PROXIMO DE 0, melhor.
dbs = metrics.davies_bouldin_score(values, labels)
print('Davies_bouldin_score {:.3f}'.format(dbs))

#Calinski,MAIS ALTO melhor.
calinski = metrics.calinski_harabasz_score(values, labels)
print('Calinski_harabasz_score {:.3f}'.format(calinski))


#Criando agrupamento de frequência por labelss
Frequencia_groupby = df[['labels','Frequencia']].groupby(by='labels')
Frequencia_groupby = Frequencia_groupby.agg(func=['count','min','mean','max']).round(2)
Frequencia_groupby = Frequencia_groupby.stack(0)
Frequencia_groupby.sort_values(['mean'], ascending=False, inplace=True)
Frequencia_groupby.reset_index(inplace=True)
Frequencia_groupby.rename(columns={'level_1': 'Freq_Col'}, inplace = True)
Frequencia_groupby.drop(['count'], axis=1, inplace=True)

#Criando agrupamento de Pallets por labelss
Venda_pallet_groupby = df[['labels','Venda_pallet']].groupby(by='labels')
Venda_pallet_groupby = Venda_pallet_groupby.agg(func=['count','min','mean','max']).round(2)
Venda_pallet_groupby = Venda_pallet_groupby.stack(0)
Venda_pallet_groupby.sort_values(['mean'], ascending=False, inplace=True)
Venda_pallet_groupby.reset_index(inplace=True)
Venda_pallet_groupby.rename(columns={'level_1': 'Pallet_Col'}, inplace = True)

#Juntando/Merge Frquencia com venda_pallet
Junção_Frequencia_groupby_Venda_pallet_groupby = Frequencia_groupby.merge(Venda_pallet_groupby, on='labels', how='left')

#Tabela final para análise de agrupamentos.
Junção_Frequencia_groupby_Venda_pallet_groupby = Junção_Frequencia_groupby_Venda_pallet_groupby.iloc[:,[0,6,1,2,4,3,5,7,9,8]]
Junção_Frequencia_groupby_Venda_pallet_groupby

#Grafico Plotly de pontinhos
AgglomerativeClustering = px.scatter(df, x=df.Venda_pallet, y=df.Frequencia, color='labels',title='AgglomerativeClustering')
AgglomerativeClustering.update_layout(width=1400, height=900,title_font_color='black',title_font_size=24)
AgglomerativeClustering.show()
AgglomerativeClustering.write_html('AgglomerativeClustering.html')

Silhouette_score 0.280
Davies_bouldin_score 1.972
Calinski_harabasz_score 2072.432


In [ ]:
Junção_Frequencia_groupby_Venda_pallet_groupby

,labels,count,Freq_Col,min_x,max_x,mean_x,Pallet_Col,min_y,max_y,mean_y
0,9,6,Frequencia,534.64,618.09,571.92,Venda_pallet,13.36,56.30,28.93
1,3,12,Frequencia,379.27,496.45,428.79,Venda_pallet,9.55,33.92,19.56
2,5,32,Frequencia,266.82,365.55,303.91,Venda_pallet,3.23,30.22,10.85
3,4,68,Frequencia,189.64,265.45,224.02,Venda_pallet,1.31,50.12,8.48
4,8,82,Frequencia,143.09,188.57,163.06,Venda_pallet,0.70,11.04,4.06
5,6,193,Frequencia,93.45,140.91,113.88,Venda_pallet,0.26,40.21,3.34
6,2,461,Frequencia,57.45,95.09,73.68,Venda_pallet,0.19,6.95,1.48
7,0,1136,Frequencia,25.73,57.27,38.61,Venda_pallet,0.06,17.49,0.75
8,1,2444,Frequencia,7.45,25.91,14.48,Venda_pallet,0.01,6.77,0.33
9,7,3569,Frequencia,1.00,8.50,3.01,Venda_pallet,0.01,13.40,0.24


###Testando outro algoritimo
####Affinity Propagation

In [ ]:
"""
#importando bibliotecas
from sklearn.cluster import AffinityPropagation
from sklearn.preprocessing import StandardScaler

#Base de dados para rodar Clusters
df = Base_dados_para_clusterizacao.copy()
ssc = StandardScaler()

#definindo e aplicando nos dados a normalização, de modo a evitar problemas por discrepancia dimensional
values = ssc.fit_transform(df.values)

#definindo o modelo de clustering que iremos utilizar
aff = AffinityPropagation(damping=0.55, max_iter=250, random_state=0)



#aplicando o algoritmo nos dados normalizados
labels = aff.fit_predict(df.values)
df['labels'] = labels.astype(str)


silhouette = metrics.silhouette_score(values, labels, metric='euclidean')
print('Silhouette_score {:.3f}'.format(silhouette))


#Davies Bouldin, MAIS PROXIMO DE 0, melhor.
dbs = metrics.davies_bouldin_score(values, labels)
print('Davies_bouldin_score {:.3f}'.format(dbs))

#Calinski,MAIS ALTO melhor.
calinski = metrics.calinski_harabasz_score(values, labels)
print('Calinski_harabasz_score {:.3f}'.format(calinski))

#Grafico Plotly de pontinhos
fig = px.scatter(df, x=df.Venda_pallet, y=df.Frequencia, color='labels')
fig.show()
"""

"\n#importando bibliotecas\nfrom sklearn.cluster import AffinityPropagation\nfrom sklearn.preprocessing import StandardScaler\n\n#Base de dados para rodar Clusters\ndf = Base_dados_para_clusterizacao.copy()\nssc = StandardScaler()\n\n#definindo e aplicando nos dados a normalização, de modo a evitar problemas por discrepancia dimensional\nvalues = ssc.fit_transform(df.values)\n\n#definindo o modelo de clustering que iremos utilizar\naff = AffinityPropagation(damping=0.55, max_iter=250, random_state=0)\n\n\n\n#aplicando o algoritmo nos dados normalizados\nlabels = aff.fit_predict(df.values)\ndf['labels'] = labels.astype(str)\n\n\nsilhouette = metrics.silhouette_score(values, labels, metric='euclidean')\nprint('Silhouette_score {:.3f}'.format(silhouette))\n\n\n#Davies Bouldin, MAIS PROXIMO DE 0, melhor.\ndbs = metrics.davies_bouldin_score(values, labels)\nprint('Davies_bouldin_score {:.3f}'.format(dbs))\n\n#Calinski,MAIS ALTO melhor.\ncalinski = metrics.calinski_harabasz_score(values, lab

In [ ]:
"""
#Criando agrupamento de frequência por labelss
Frequencia_groupby = df[['labels','Frequencia']].groupby(by='labels')
Frequencia_groupby = Frequencia_groupby.agg(func=['count','min','mean','max']).round(2)
Frequencia_groupby = Frequencia_groupby.stack(0)
Frequencia_groupby.sort_values(['mean'], ascending=False, inplace=True)
Frequencia_groupby.reset_index(inplace=True)
Frequencia_groupby.rename(columns={'level_1': 'Freq_Col'}, inplace = True)
Frequencia_groupby.drop(['count'], axis=1, inplace=True)

#Criando agrupamento de Pallets por labelss
Venda_pallet_groupby = df[['labels','Venda_pallet']].groupby(by='labels')
Venda_pallet_groupby = Venda_pallet_groupby.agg(func=['count','min','mean','max']).round(2)
Venda_pallet_groupby = Venda_pallet_groupby.stack(0)
Venda_pallet_groupby.sort_values(['mean'], ascending=False, inplace=True)
Venda_pallet_groupby.reset_index(inplace=True)
Venda_pallet_groupby.rename(columns={'level_1': 'Pallet_Col'}, inplace = True)

#Juntando/Merge Frquencia com venda_pallet
Junção_Frequencia_groupby_Venda_pallet_groupby = Frequencia_groupby.merge(Venda_pallet_groupby, on='labels', how='left')

#Tabela final para análise de agrupamentos.
Junção_Frequencia_groupby_Venda_pallet_groupby = Junção_Frequencia_groupby_Venda_pallet_groupby.iloc[:,[0,6,1,2,4,3,5,7,9,8]]

Junção_Frequencia_groupby_Venda_pallet_groupby
"""

"\n#Criando agrupamento de frequência por labelss\nFrequencia_groupby = df[['labels','Frequencia']].groupby(by='labels')\nFrequencia_groupby = Frequencia_groupby.agg(func=['count','min','mean','max']).round(2)\nFrequencia_groupby = Frequencia_groupby.stack(0)\nFrequencia_groupby.sort_values(['mean'], ascending=False, inplace=True)\nFrequencia_groupby.reset_index(inplace=True)\nFrequencia_groupby.rename(columns={'level_1': 'Freq_Col'}, inplace = True)\nFrequencia_groupby.drop(['count'], axis=1, inplace=True)\n\n#Criando agrupamento de Pallets por labelss\nVenda_pallet_groupby = df[['labels','Venda_pallet']].groupby(by='labels')\nVenda_pallet_groupby = Venda_pallet_groupby.agg(func=['count','min','mean','max']).round(2)\nVenda_pallet_groupby = Venda_pallet_groupby.stack(0)\nVenda_pallet_groupby.sort_values(['mean'], ascending=False, inplace=True)\nVenda_pallet_groupby.reset_index(inplace=True)\nVenda_pallet_groupby.rename(columns={'level_1': 'Pallet_Col'}, inplace = True)\n\n#Juntando/Me